# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 6. Support Vector Machine (SVM)


- Nama: Rayhan Egar Sadtya Nugraha
- NIM: 225150201111014
- Kelas: PPM-D

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [47]:
from sklearn.datasets import load_iris
iris_data = load_iris()
X_iris = iris_data.data
y_iris = iris_data.target
col_iris = iris_data.feature_names 

Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.DataFrame(data=X_iris, columns=col_iris)
data.rename(columns={'sepal length (cm)':'sepal length', 'sepal width (cm)':'sepal width', 'petal length (cm)':'petal length', 'petal width (cm)':'petal width'}, inplace=True)
data['variety'] = y_iris
data['variety'] = data['variety'].map({0:'Iris-setosa', 1:'Iris-versicolor', 2:'Iris-virginica'})



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [49]:
data.head()

,sepal length,sepal width,petal length,petal width,variety
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Metode Support Vector Machine (SVM) dasar hanya mampu melakukan klasifikasi data yang terdiri dari dua kelas, atau disebut klasifikasi biner. Dataset iris memiliki 3 kelas, sehingga salah satu kelasnya, yaitu Iris-virginica perlu dihapus. Cell berikut menghapus data yang memiliki kelas Iris-virginica

In [50]:
virginica_index = data[data['variety']=='Iris-virginica'].index
data1 = data.drop(virginica_index, axis=0)
data1['variety'].value_counts()

variety
Iris-setosa        50
Iris-versicolor    50
Name: count, dtype: int64

SVM juga memiliki keterbatasan, yaitu hanya mampu menerima kelas dalam bentuk numerik. Cell berikut mengubah kelas menjadi bilangan, Iris setosa menjadi -1 dan Iris-versicolor menjadi 1

In [51]:
data1['variety'] = data1['variety'].map({'Iris-setosa':-1, 'Iris-versicolor':1})

Tampilkan beberapa data untuk mengecek hasilnya

In [52]:
data1.head()

,sepal length,sepal width,petal length,petal width,variety
0,5.1,3.5,1.4,0.2,-1
1,4.9,3.0,1.4,0.2,-1
2,4.7,3.2,1.3,0.2,-1
3,4.6,3.1,1.5,0.2,-1
4,5.0,3.6,1.4,0.2,-1


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [53]:
from sklearn.model_selection import train_test_split
data1_train, data1_test = train_test_split(data1, test_size=0.2, stratify=data1['variety'], random_state=0)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 80 data, dan **data_uji** terdiri dari 20 data

In [54]:
print("Training data shape: ", data1_train.shape)
print("Testing data shape: ", data1_test.shape)
print("Training data target: ", data1_train['variety'].value_counts())
print("Testing data target: ", data1_test['variety'].value_counts())

Training data shape:  (80, 5)
Testing data shape:  (20, 5)
Training data target:  variety
 1    40
-1    40
Name: count, dtype: int64
Testing data target:  variety
 1    10
-1    10
Name: count, dtype: int64


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_latih** dan **label_uji**

In [55]:
X_train = data1_train[data1_train.columns[:-1]]
y_train = data1_train[data1_train.columns[-1]]

X_test = data1_test[data1_test.columns[:-1]]
y_test = data1_test[data1_test.columns[-1]]

In [56]:
X_train.head()
y_train.head()

60    1
13   -1
7    -1
82    1
20   -1
Name: variety, dtype: int64

## 3) Proses Training

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [57]:
def hitung_cost_gradient(W, X, Y, regularization):
    jarak = 1 - (Y * np.dot(X, W))
    dw = np.zeros(len(W))
    if max(0, jarak)==0:
        di = W
    else:
        di = W - (regularization * Y * X)
    dw += di
    return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD

In [58]:
from sklearn.utils import shuffle

def sgd(data_latih, label_latih, learning_rate=1e-6, max_epoch=1000, regularization=10000):
    data_latih = data_latih.to_numpy()
    label_latih = label_latih.to_numpy()
    bobot = np.zeros(data_latih.shape[1])

    for epoch in range(1, max_epoch):
        X, Y = shuffle(data_latih, label_latih, random_state=101)
        for index, x in enumerate(X):
            delta = hitung_cost_gradient(bobot, x, Y[index], regularization)
            bobot = bobot - (learning_rate * delta)

    return bobot

Proses training dilakukan dengan memanggil fungsi **sgd** dengan parameter input berupa data latih dan label latih. Parameter learning rate dan max epoch menggunakan nilai *default* nya

In [59]:
W = sgd(X_train, y_train)
print(W)

[-0.21288919 -0.72751185  1.112969    0.54579068]


## 4) Proses *testing*
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [60]:
def testing(W, data_uji):
    prediksi = np.array([])
    for i in range(data_uji.shape[0]):
        y_prediksi = np.sign(np.dot(W, data_uji.to_numpy()[i]))
        prediksi = np.append(prediksi, y_prediksi)
    return prediksi

Lakukan pengujian menggunakan seluruh data uji. Bandingkan hasilnya dengan nilai label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar.

In [61]:
y_prediksi = testing(W, X_test)
print(sum(y_prediksi==y_test))

20


## TUGAS
Pada tugas kali ini Anda masih melakukan klasifikasi dataset iris, tetapi menggunakan kelas Iris-versicolor dan Iris-virginica. Apakah hasilnya lebih baik jika dibandingkan dengan Iris-setosa dan Iris-versicolor?

In [62]:
setosa_index = data[data['variety']=='Iris-setosa'].index
data2 = data.drop(setosa_index, axis=0)
print("data2 target: ", data2['variety'].value_counts())
data2['variety'] = data2['variety'].map({'Iris-virginica':-1, 'Iris-versicolor':1})

data2_train, data2_test = train_test_split(data2, stratify=data2['variety'], test_size=.2, random_state=0)

X_train_2 = data2_train[data2_train.columns[:-1]]
y_train_2 = data2_train[data2_train.columns[-1]]
X_test_2 = data2_test[data2_test.columns[:-1]]
y_test_2 = data2_test[data2_test.columns[-1]]

print("Training data shape: ", X_train_2.shape)
print("Testing data shape: ", X_test_2.shape)

data2 target:  variety
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64
Training data shape:  (80, 4)
Testing data shape:  (20, 4)


In [63]:
W2 = sgd(X_train_2, y_train_2)
print(W2)

[ 3.02332426  3.67643787 -4.12106681 -5.33778464]


In [64]:
y_prediksi_2 = testing(W2, X_test_2)
print(sum(y_prediksi_2==y_test_2))

20


Hasilnya tidak lebih baik karena hanya mampu mengenali 17 diantara 20 data uji.

Lakukan proses training ulang, tetapi menggunakan nilai max_epoch sebesar 5000. Anda dapat memberikan parameter max_epoch=5000 saat memanggil fungsi **sgd**. Apakah hasilnya lebih baik? Mengapa?

In [66]:
W2 = sgd(X_train_2, y_train_2, max_epoch=5000)
print(W2)
y_prediksi_2 = testing(W2, X_test_2)
print(sum(y_prediksi_2==y_test_2))

[ 3.02102109  3.7106945  -4.26603033 -5.42614294]
20


Terakhir, lakukanlah seleksi fitur sederahana dengan mengambil hanya fitur sepal length dan sepal width saja. Apakah hasilnya lebih baik?

In [67]:
X_train_2.drop(columns=['petal length', 'petal width'], inplace=True)
X_test_2.drop(columns=['petal length', 'petal width'], inplace=True)

/tmp/ipykernel_11703/1818913257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_2.drop(columns=['petal length', 'petal width'], inplace=True)
/tmp/ipykernel_11703/1818913257.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_2.drop(columns=['petal length', 'petal width'], inplace=True)


In [75]:
W2_feat_select = sgd(X_train_2, y_train_2, max_epoch=5000)
print(W2_feat_select)

[-0.88113272  1.82943048]


In [76]:
y_prediksi_2 = testing(W2_feat_select, X_test_2)
print(sum(y_prediksi_2==y_test_2))

8


# Penjelasan